In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:

img_orig = cv2.imread('Fotos 1-11-097.jpeg', cv2.COLOR_BGR2RGB) 
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)
img_grey = cv2.imread('Fotos 1-11-097.jpeg', 0)                         # el cero final es para que la lea en greyscale



In [ ]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
img_orig_b = increase_brightness(img_orig, value=30)
plt.figure(figsize = (200,20))
plt.imshow(img_orig_b) 

In [ ]:
plt.figure(figsize = (200,20))
plt.imshow(img_orig) 

In [ ]:
# eye detection

lower_hsv = np.array([11, 40, 38])
higher_hsv = np.array([5, 38, 36])

mask = cv2.inRange(img_orig, cv2.bgr(1,1,1), higher_hsv)

detected_output = cv2.bitwise_and(img_orig, img_orig, mask = mask)

plt.figure(figsize = (200,20))
plt.imshow( detected_output) 


In [ ]:
img_orig_b = increase_brightness(img_orig, value=150)
img_grey  = cv2.cvtColor(img_orig_b, cv2.COLOR_BGR2GRAY)
plt.figure(figsize = (200,20))
plt.imshow(img_grey) 

In [ ]:
img_ = cv2.GaussianBlur(img_grey, (3,3), 0)
_, img_thr = cv2.threshold(img_, 230, 255, cv2.THRESH_BINARY)

#img_thr = cv2.adaptiveThreshold(img_, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 5)
#_, img_thr = cv2.threshold(img_, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

plt.figure(figsize = (200,20))
plt.imshow(img_thr, cmap="gray") 

In [ ]:
image = img_orig

b = image.copy()
# set green and red channels to 0
b[:, :, 1] = 0
b[:, :, 2] = 0


g = image.copy()
# set blue and red channels to 0
g[:, :, 0] = 0
g[:, :, 2] = 0

r = image.copy()
# set blue and green channels to 0
r[:, :, 0] = 0
r[:, :, 1] = 0


# RGB - Blue


plt.figure(figsize = (200,20))
plt.imshow(b, cmap="gray") 

# RGB - Green

plt.figure(figsize = (200,20))
plt.imshow(g, cmap="gray") 

# RGB - Red
plt.figure(figsize = (200,20))
plt.imshow(r, cmap="gray") 


In [ ]:
# dilatation

kernel = np.ones((3,3), np.uint8)



img_thr_dil = cv2.dilate(img_thr, kernel, iterations = 7)
img_end = cv2.erode(img_thr_dil, kernel, iterations = 7)

plt.figure(figsize = (200,20))
plt.imshow(img_end, cmap="gray") 

In [ ]:
# invert ... para opencv blanco es el obj
img_thr = cv2.bitwise_not(img_thr)

In [ ]:
# dilatation

kernel = np.ones((3,3), np.uint8)

img_thr_ero = cv2.erode(img_thr, kernel, iterations = 7)
img_thr_dil = cv2.dilate(img_thr_ero, kernel, iterations = 12)

plt.figure(figsize = (200,20))
plt.imshow(img_thr_dil, cmap="gray") 

In [ ]:
# HAY 84 MOSCAS
t  = 18
cont = 0
contornos, _ = cv2.findContours(img_thr_dil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(contornos))
for c in contornos:
    if len(c)>t: 
        #print(len(c))
        cont = cont + 1

print("cant contornos", cont)

In [ ]:
#img_fin = img_orig
img_fin = img_thr_dil
img_fin2 = img_orig

hmin = 20
wmin = 20
cont = 0

hmax = 120
wmax = 120
rect = 150

for c in contornos:
    if len(c)>t:
        (x, y, w, h) = cv2.boundingRect(c)
        if ((h>hmin) and (w>wmin)) and ((h<hmax) and (w<wmax)):

            x0 = int((x + w/2) - (rect/2))
            y0 = int((y + h/2) - (rect/2))
            cont=cont+1

            #cv2.rectangle(img_thr, (x0, y0), (x0+rect, y0+rect), (255, 255, 255), 5)
            cv2.rectangle(img_fin, (x0, y0), (x0+rect, y0+rect), (255, 0, 255), 5)
            cv2.rectangle(img_fin2, (x0, y0), (x0+rect, y0+rect), (255, 0, 255), 5)
        else:
            cont=cont+1
            cv2.rectangle(img_fin, (x0, y0), (x0+rect, y0+rect), (0, 255, 255), 5)
            cv2.rectangle(img_fin2, (x0, y0), (x0+rect, y0+rect), (0, 255, 255), 5)
        

print(cont)

In [ ]:
plt.figure(figsize = (200,20))
plt.imshow(img_fin)

In [ ]:
plt.figure(figsize = (200,20))
plt.imshow(img_fin2)

In [ ]:
from skimage.filters import threshold_local

img_ = cv2.GaussianBlur(img_orig, (25,25), 0)

V = cv2.split(cv2.cvtColor(img_, cv2.COLOR_BGR2HSV))[2]
T = threshold_local(V, 127, offset=7, method="gaussian")

thresh = (V > T).astype("uint8") * 255

plt.figure(figsize = (200,20))
plt.imshow(thresh)


In [ ]:
plt.figure(figsize = (100,10))
plt.imshow(img_grey, cmap='gray') #"Grises"

In [ ]:
mask = np.zeros([2848, 4288], np.int8)
cv2.circle(mask,(2540, 1050), 1500, (255,255,255), -1)

plt.imshow(mask, cmap='gray')

print(mask[1500,1030])
print(mask[0,0])


img_grey[mask == 0] = 255


In [ ]:
_, th = cv2.threshold(img_grey, 120, 255, cv2.THRESH_BINARY)

plt.figure(figsize = (100,10))
plt.imshow(th)


In [ ]:
contornos, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(contornos))

In [ ]:
img_bb = img_orig
hmax = 150
wmax = 150
rect = 150

for c in contornos:
    (x, y, w, h) = cv2.boundingRect(c)
    #if (h<hmax) or (w<wmax):

    x0 = int((x + w/2) - (rect/2))
    y0 = int((y + h/2) - (rect/2))

    cv2.rectangle(img_bb, (x0, y0), (x0+rect, y0+rect), (0, 0, 255), 2)
    #else:
    #    cv2.rectangle(img_bb, (x, y), (x+w, y+h), (0, 0, 255), 20)

plt.figure(figsize = (200,20))
plt.imshow(img_bb)